In [1]:
import graphlab

In [2]:
graphlab.canvas.set_target('ipynb')

In [3]:
taps = graphlab.SFrame.read_csv("taps.csv", verbose=False)

This non-commercial license of GraphLab Create for academic use is assigned to jenarvaezg@gmail.com and will expire on December 12, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1503231740.log


In [4]:
features = [
    "n_accel",
    
    "accel_x_mean",
    "accel_x_median",
    "accel_x_var",
    "accel_x_skewness",
    "accel_x_kurtosis",
    "accel_x_diff",
    
    "accel_y_mean",
    "accel_y_median",
    "accel_y_skewness",
    "accel_y_kurtosis",
    "accel_y_var",
    "accel_y_diff",
    
    "accel_z_mean",
    "accel_z_var",   
    "accel_z_median",
    "accel_z_skewness",
    "accel_z_kurtosis",
    "accel_z_diff",
    
    "n_gyro",
    
    "gyro_x_mean",
    "gyro_x_median",
    "gyro_x_var",
    "gyro_x_skewness",
    "gyro_x_kurtosis",
    "gyro_x_diff",
    
    "gyro_y_mean",
    "gyro_y_median",
    "gyro_y_var",
    "gyro_y_skewness",
    "gyro_y_kurtosis",
    "gyro_y_diff",
    
    "gyro_z_mean",
    "gyro_z_median",
    "gyro_z_var",
    "gyro_z_skewness",
    "gyro_z_kurtosis",
    "gyro_z_diff",
    
] 



In [5]:
train, test = taps.random_split(0.8, seed=1)

In [6]:
noise_model = graphlab.load_model("noise_model")
type_model = graphlab.load_model("type_model")
touch_model = graphlab.load_model("touch_model")
swipe_model = graphlab.load_model("swipe_model")

In [7]:
noise = test[test['noise'] == 1]
not_noise = test[test['noise'] == 0]

touches = not_noise[not_noise['type'] == "TOUCH"]
swipes = not_noise[not_noise['type'] == "SWIPE"]

# [Ceiling Analysis explained](https://d3c33hcgiwev3.cloudfront.net/19.4-ApplicationExamplePhotoOCR-CeilingAnalysisWhatPartOfThePipelineToWorkOnNext.6acb6550b22b11e49c064db6ead92550/full/540p/index.mp4?Expires=1488067200&Signature=Wlu5B9uRkREYKt0~6bVR~i6Tp5zN6k-uCRjFczQZKDu1dy6a1R55DLNWtknej1AbqtmZccxk8qDUiIJ7kT9U48llZxK1Y1BV2jqHR~iSkYK-pLmqZDCH-bDkEyMgDj-bdckYvqadcFpKj308QhIW8ihdsql-h9ehQx5DlDG5ERs_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A)

# Full model

In [8]:
predicted_not_noise = test[noise_model.predict(test) == 0]


predicted_touches = predicted_not_noise[type_model.predict(predicted_not_noise) == "TOUCH"]
predicted_swipes = predicted_not_noise[type_model.predict(predicted_not_noise) == "SWIPE"]
touch_evaluation = touch_model.evaluate(predicted_touches)
swipe_evaluation = swipe_model.evaluate(predicted_swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 62.7298311826 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  1156 |
|  CENTER-RIGHT |   CENTER-RIGHT  |  1153 |
|  CENTER-LEFT  |   CENTER-LEFT   |  1137 |
|    TOP-LEFT   |     TOP-LEFT    |  1130 |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  1129 |
|   TOP-RIGHT   |    TOP-RIGHT    |  1107 |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  1099 |
|     CENTER    |      CENTER     |  1089 |
|   TOP-CENTER  |    TOP-CENTER   |  1043 |
|     NOISE     |   BOTTOM-LEFT   |   49  |
+---------------+-----------------+-------+
[64 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 95.9216809933 %
Swipe final f1 35.9642778436 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+----

# When providing real not_noise

In [9]:
predicted_touches = not_noise[type_model.predict(not_noise) == "TOUCH"]
predicted_swipes = not_noise[type_model.predict(not_noise) == "SWIPE"]
touch_evaluation = touch_model.evaluate(predicted_touches)
swipe_evaluation = swipe_model.evaluate(predicted_swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 68.472966231 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  1200 |
|  CENTER-RIGHT |   CENTER-RIGHT  |  1181 |
|  CENTER-LEFT  |   CENTER-LEFT   |  1176 |
|    TOP-LEFT   |     TOP-LEFT    |  1156 |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  1152 |
|   TOP-RIGHT   |    TOP-RIGHT    |  1137 |
|     CENTER    |      CENTER     |  1123 |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  1119 |
|   TOP-CENTER  |    TOP-CENTER   |  1078 |
|    TOP-LEFT   |    TOP-CENTER   |   11  |
+---------------+-----------------+-------+
[63 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 98.7845726864 %
Swipe final f1 39.8129585912 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+-----

# When providing real swipes and touches

In [10]:
touch_evaluation = touch_model.evaluate(touches)
swipe_evaluation = swipe_model.evaluate(swipes)

print "Touch final f1", touch_evaluation['f1_score']*100, "%"
print "Touch final confusion"
print touch_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Touch final accuracy", touch_evaluation['accuracy']*100, "%"

print "Swipe final f1", swipe_evaluation['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_evaluation['confusion_matrix'].sort("count", ascending=False)
print "Swipe final accuracy", swipe_evaluation['accuracy']*100, "%"

Touch final f1 98.9466760418 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  1202 |
|  CENTER-RIGHT |   CENTER-RIGHT  |  1190 |
|  CENTER-LEFT  |   CENTER-LEFT   |  1176 |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  1159 |
|    TOP-LEFT   |     TOP-LEFT    |  1156 |
|   TOP-RIGHT   |    TOP-RIGHT    |  1140 |
|     CENTER    |      CENTER     |  1128 |
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  1119 |
|   TOP-CENTER  |    TOP-CENTER   |  1078 |
|    TOP-LEFT   |    TOP-CENTER   |   11  |
+---------------+-----------------+-------+
[48 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Touch final accuracy 98.948173647 %
Swipe final f1 99.6726103217 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+-----